<img src="images/dask_horizontal.svg" align="right" width="30%">

# 分布式执行

正如我们目前所看到的，Dask 允许您简单地构建具有依赖性的任务图，以及使用函数式、Numpy 或 Pandas 语法在数据集合上自动为您创建图形。如果没有一种并行和内存感知的方式来执行这些图，这些都不会很有用。到目前为止，我们一直在调用`thing.compute()`或`dask.compute(thing)`，而不用担心这会带来什么。现在我们将讨论该执行的可用选项，特别是分布式调度器，它带有额外的功能。

Dask自带四个可用的调度器。
- "线程"（又名 "线程"）：由线程池支持的调度器
- "进程"：一个由进程池支持的调度器；
- "单线程"（又名 "同步"）：同步调度器，适合调试
- distributed：用于在多台机器上执行图形的分布式调度器，见下文。

要选择其中一个进行计算，你可以在请求结果时指定，例如：
```python
myvalue.compute(scheduler="single-threaded")  # for debugging
```

您也可以临时设置一个默认的调度程序。
```python
with dask.config.set(scheduler='processes'):
    # 只为该块临时设置
    # 该块中的所有计算调用都将使用指定的调度器。
    myvalue.compute()
    anothervalue.compute()
```

Or globally
```python
# set until further notice
dask.config.set(scheduler='processes')
```

让我们在航班数据这个熟悉的案例上，试用几个调度器。

In [ ]:
%run prep.py -d flights

In [ ]:
import dask.dataframe as dd
import os
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

# 按机场分类的最大平均非取消航班延误情况
largest_delay = df[~df.Cancelled].groupby('Origin').DepDelay.mean().max()
largest_delay

In [ ]:
# 下面的每一个结果都是一样的(你可以检查一下)
# any surprises?
import time
for sch in ['threading', 'processes', 'sync']:
    t0 = time.time()
    r = largest_delay.compute(scheduler=sch)
    t1 = time.time()
    print(f"{sch:>10}, {t1 - t0:0.4f} s; result, {r:0.2f} hours")

### 一些需要考虑的问题：

- 这个任务有可能提速多少（提示，看图）。
- 考虑到这台机器上有多少核，并行调度器比单线程调度器能快多少。
- 使用线程比单线程快多少？为什么这与最佳提速有差异？
- 为什么这里多处理调度器会慢这么多？

对于在单机上处理大型数据集的核心外，只要所使用的函数在大部分时间释放[GIL](https://wiki.python.org/moin/GlobalInterpreterLock)，"线程 "调度器是一个不错的选择。NumPy和pandas在大多数地方都会释放GIL，所以对于`dask.array`和`dask.dataframe`来说，`threaded`调度器是默认的。分布式调度器，或许用`processes=False`，对于单机上的这些工作负载也能很好地工作。

对于确实持有GIL的工作负载，比如常见的`dask.bag`和用`dask.delayed`封装的自定义代码，我们建议使用分布式调度器，即使在单机上也是如此。一般来说，它比`processes`调度器更智能，提供更好的诊断。

https://docs.dask.org/en/latest/scheduling.html 提供了一些关于选择调度器的额外细节。

对于在集群上扩展工作，需要使用分布式调度器。

## 创建集群

### 简化方法

`dask.distributed`系统由一个集中式调度器和一个或多个工作进程组成。[部署](https://docs.dask.org/en/latest/setup.html)一个远程Dask集群涉及一些额外的工作。但在本地做事情只是涉及到创建一个`Client`对象，它让你与 "集群"（你的机器上的本地线程或进程）交互。更多信息请参见[这里](https://docs.dask.org/en/latest/setup/single-distributed.html)。

请注意，`Client()`需要很多可选的[参数](https://distributed.dask.org/en/latest/local-cluster.html#api)，来配置进程/线程的数量、内存限制和其他的参数。

In [ ]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client.cluster

如果你没有在jupyterlab中使用 "dask-labextension"，一定要点击 "Dashboard "链接来打开诊断仪表板。

## 用分布式客户端执行

考虑一些琐碎的计算，比如我们以前用过的，为了模拟真实工作的进行，我们加入了睡眠语句。

In [ ]:
from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

默认情况下，创建一个 "客户端 "使它成为默认的调度器。任何对`.compute`的调用都将使用你的`client`所连接的集群，除非你如上所述另有规定。


In [ ]:
x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
total.compute()

当任务被集群处理时，它们会出现在web UI中，最终，结果会被打印出来，作为上面单元格的输出。请注意，在等待结果的过程中，内核是被阻塞的。结果的任务块图可能看起来像下面的东西。将鼠标悬停在每个块上，会给出与之相关的函数，以及它执行的时间。![this](images/tasks.png)

只要计算是在飞行中，您还可以在仪表板的 "图形 "面板上看到正在执行的简化版图形。

让我们回到之前的航班计算，看看仪表盘上会发生什么（你可能希望笔记本和仪表盘并排）。与之前相比，表现如何？

In [ ]:
%time largest_delay.compute()

在这种特殊情况下，这应该和上面的最佳情况，即线程一样快，甚至更快。你认为这是为什么呢？你应该从阅读[这里](https://distributed.dask.org/en/latest/index.html#architecture)开始，特别要注意的是，分布式调度器是一个完整的重写，围绕着中间结果的共享和哪些任务在哪个worker上运行，有了更多的智能。这将在*些情况下带来更好的性能，但与线程式调度器相比，仍然有较大的延迟和开销，所以会有极少数情况下性能更差。幸运的是，现在仪表板给我们提供了更多的 [诊断信息](https://distributed.dask.org/en/latest/diagnosing-performance.html) 。在仪表板的Profile页面，看看我们刚刚进行的计算中，什么东西占用了CPU时间的最大部分？

如果你想做的只是执行使用延迟创建的计算，或者运行基于上层数据集合的计算，那么这就是你需要知道的所有内容，以将你的工作扩展到集群规模。然而，关于分布式调度器还有更多细节需要了解，这将有助于高效使用。请参阅《分布式，高级》一章。

### 练习

在查看诊断页面的同时，运行以下计算。在每一种情况下，什么东西花费的时间最长？

In [ ]:
# 航班数
_ = len(df)

In [ ]:
# 未取消的航班数量
_ = len(df[~df.Cancelled])

In [ ]:
# 每个机场未被取消的航班数量
_ = df[~df.Cancelled].groupby('Origin').Origin.count().compute()

In [ ]:
# 每个机场的平均起飞延误时间？
_ = df[~df.Cancelled].groupby('Origin').DepDelay.mean().compute()

In [ ]:
# 每周一天的平均出发延误
_ = df.groupby(df.Date.dt.dayofweek).DepDelay.mean().compute()

In [ ]:
client.shutdown()